In [1]:
import numpy as np
import os
import sys
import warnings
import time

measure_sampling_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Measure_sampling')
trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Trees')

if measure_sampling_path not in sys.path:
    sys.path.append(measure_sampling_path)

if trees_path not in sys.path:
    sys.path.append(trees_path)

print("Paths added to sys.path!")

# Now import
from BM import generate_adapted_tree_from_brownian
from Save_Load_trees import save_tree, load_tree


# Suppress sklearn warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Set parameters
num_paths = 3000   # number of sample paths
time_steps = 4   # number of time steps per path
scale = 1
use_weights = 1

# Generate the adapted tree from Brownian motion with timing
print("Generating adapted tree from Brownian motion...")
tree_root, time_tdetail = generate_adapted_tree_from_brownian(num_paths=num_paths, time_steps=time_steps, scale=scale, use_weights=use_weights, return_times=True)

print(f"Adapted tree generated successfully in {sum(time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {time_tdetail[0]:.2f}s, Measure adaptation: {time_tdetail[1]:.2f}s, Tree construction: {time_tdetail[2]:.2f}s")

Paths added to sys.path!
Generating adapted tree from Brownian motion...
Adapted tree generated successfully in 27.34 seconds.
Time breakdown: Sample generation: 0.02s, Measure adaptation: 27.24s, Tree construction: 0.09s


In [2]:
tree_2_root = load_tree('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Trees/Data_trees_exemple', 'tree_2.pkl')
visualize_tree(tree_2_root, title="emp gaus")

NameError: name 'load_tree' is not defined

In [5]:
def print_results(distance, prob_matrix, elapsed_time):
    """Prints the nested distance, probability matrix, and computation time in a formatted way."""
    print(f"Nested Distance: {distance}\n")
    print(f"Final Probability Matrix:")
    for row in prob_matrix:
        print(" | ".join(f"{value:.5f}" for value in row))
    print(f"\nComputation Time: {elapsed_time:.4f} seconds\n")
    print("-" * 50 + "\n")

In [ ]:
import sys
import os
import time

# Define the absolute path to the AWD_trees folder
awd_trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/AWD_trees')

# Add it to sys.path if not already present
if awd_trees_path not in sys.path:
    sys.path.append(awd_trees_path)

import time
from AWD_from_comp_methodes_AOT import *

formatted_tree_1 = get_sample_paths(tree_root)
formatted_tree_2 = get_sample_paths(tree_2_root)

def cost_function(x, y):
    """Cost function: L1 distance."""
    return np.sum(np.abs(x - y))

start_time = time.time()

val_gurobi = gurobi_bm(
    [formatted_tree_1, formatted_tree_2],
    f=cost_function,
    r_opti=0,
    causal=1,
    anticausal=1,
    outputflag=0
)

end_time = time.time()
elapsed_time_gurobi = end_time - start_time


print(val_gurobi, elapsed_time_gurobi)

In [ ]:
import time

from Algorithm import compute_nested_distance

# Start timing the Sinkhorn with regularization computation


start_time = time.time()

max_depth = get_depth(tree_root)

start_time = time.time()

distance = compute_nested_distance(
    tree_root, tree_2_root, max_depth, method="solver_lp", return_matrix=False, lambda_reg=0
)

end_time = time.time()
elapsed_time = end_time - start_time

print(distance)
print(elapsed_time)

In [ ]:
start_time = time.time()

distance = compute_nested_distance(
    tree_root, tree_2_root, max_depth, method="solver_pot", return_matrix=False, lambda_reg=0
)

end_time = time.time()
elapsed_time = end_time - start_time

print(distance)
print(elapsed_time)

In [ ]:
# Start timing the Sinkhorn with regularization computation
start_time = time.time()

# Compute nested distance using Sinkhorn iteration with regularization
distance_sinkhorn_reg = compute_nested_distance(
    tree_root, tree_2_root, max_depth, method="Sinkhorn", return_matrix=False, lambda_reg=2
)

# End timing
end_time = time.time()
elapsed_time_sinkhorn_reg = end_time - start_time

print(distance)
print(elapsed_time_sinkhorn_reg)